In [1]:
import sqlite3
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

In [2]:
conn = sqlite3.connect('yelp_dataset_business.db') 
df_business= pd.read_sql("""SELECT * FROM business""",conn )
conn.close()

In [3]:
df_business.is_open = df_business.is_open.astype(int)
df_business["review_count"]= df_business["review_count"].astype(int)
df_business['attributes']= df_business['attributes'].apply(lambda x: eval(x) if x else None)
df_business['hours']= df_business['hours'].apply(lambda x: eval(x) if x else None)

In [4]:
conn = sqlite3.connect('yelp_dataset_reviews.db') 
df_reviews= pd.read_sql("""SELECT ubt.*
                        FROM reviews ubt
                        INNER JOIN (
                         SELECT user_id, business_id, MAX(date) as max_date
                        FROM reviews
                        GROUP BY 1,2
) grouped_ubt ON ubt.user_id = grouped_ubt.user_id
              AND ubt.business_id = grouped_ubt.business_id
              AND ubt.date = grouped_ubt.max_date;
""",conn )
conn.close()

In [5]:
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31


In [12]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [13]:
select_feature = df_reviews[['review_id','user_id','business_id','stars']]
business_fea = df_business[['business_id', 'name', 'city', 'state','stars', 'is_open','categories']]

In [17]:
df_merge = pd.merge(select_feature, business_fea,suffixes=('_review', '_business'), on='business_id', how='inner')

In [22]:
df_merge.dtypes

review_id          string
user_id            string
business_id        string
stars_review      float64
name               string
city               string
state              string
stars_business    float64
is_open             int32
categories         string
dtype: object

In [21]:
convert_dict = {'review_id': "string",
                'user_id' : "string", 
                'business_id': "string", 
                'name': "string",
                'city': "string",
                'state': "string",
                'categories': "string"
                }
 
df_merge = df_merge.astype(convert_dict)

In [28]:
busReview_schema="""
DROP TABLE IF EXISTS "busReview";

CREATE TABLE "busReview" (
    "review_id" VARCHAR PRIMARY KEY NOT NULL,
    "user_id" VARCHAR,
    "business_id" VARCHAR ,
    "stars_review" FLOAT,
    "name" VARCHAR,
    "address" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "stars_business" FLOAT,
    "is_open" BOOLEAN,
    "categories" VARCHAR
    );
"""

In [29]:
conn = sqlite3.connect('yelp_dataset_businessReview.db') 
c = conn.cursor()
c.executescript(busReview_schema)
conn.commit()

In [30]:
df_merge.to_sql(name='busReview', con=conn, if_exists='append',index=False)
conn.close()